[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danielegrattarola/ml-18-19/blob/master/01_linear_regression.ipynb)

# Machine Learning

Prof. Cesare Alippi  
Daniele Grattarola  ([`daniele.grattarola@usi.ch`](mailto:daniele.grattarola@usi.ch)  )    
Daniele Zambon ([`daniele.zambon@usi.ch`](mailto:daniele.zambon@usi.ch)  )

---
# Lab 01: Linear Regression

System model
$$y = \beta_1  x_1 + \beta_2  x_2 + \dots + \beta_d  x_d + \varepsilon = \mathbf x^\top \beta +\varepsilon$$

Data set of $n$ observations
$$\{(\mathbf x_1, y_1), (\mathbf x_2, y_2) ,\dots,(\mathbf x_n, y_n)\}.$$

Linear regression model 
$$
\begin{cases}
y = \mathbf x^\top \beta +\varepsilon\\
\varepsilon \sim N(0, \sigma^2)\quad i.i.d.
\end{cases}
$$

**Q)** Given an observation $\mathbf x$, what is our best prediction for $y$?

**A)** If we know $\beta$ then 
$$\hat y = E[y] = \mathbf x^\top \beta + E[\varepsilon] = \mathbf x^\top \beta.$$

Here is how we proceed:
1. We estimate $\beta$ on a training set by minimising 
$$\hat \beta = {\rm arg}{\min}_{\beta} \sum_{i=1}^n \lVert y_i - \mathbf x_i^\top \beta \rVert^2_2$$
2. We predict new responses with 
$$\hat y = \mathbf x^\top \hat \beta.$$


## More generally

* The response $y$ can be a vector $\mathbf y\in\mathbb{R}^m$;
* The regressor can be of the form 
$$\mathbf x' = [f_1(\mathbf x), f_2(\mathbf x), \dots, f_d(\mathbf x)]^\top$$ 

We result in 
$$\mathbf y = \beta_1 f_1(\mathbf x) + \beta_2 f_d(\mathbf x) + \dots + \beta_d f_d(\mathbf x)$$

## 01.A: Ordinary Linear Regression

Consider the univariate case $f : \mathbb{R} \rightarrow \mathbb{R}$, we all know that functions of the form
$$ f(x; \beta) = \beta_1 x + \beta_0, \quad \beta_1, \beta_0\in\mathbb{R}$$
are all lines in the plane.

Define in python the function
$$ f(x) = \tfrac{3}{10} x + 2 $$

In [0]:
def lin_fun(x):
    y = 0.3 * x + 2
    return y



Plot the graph of the function

In [0]:
import numpy as np
import matplotlib.pyplot as plt

# create n observations
n = 11 
x = np.linspace(-1, 1, n)
print('x = {}'.format(x))

# plot
plt.plot(x, lin_fun(x), '*-');

### Noise perturbance

Consider a model affected by Gaussian noise: $$ y = f(x) + \varepsilon, \quad\varepsilon\sim \mathcal{N}(0, \sigma^2)$$

Generate $n$ observations: $$\{(x_1,y_1),(x_2,y_2),\dots,(x_n,y_n)\}$$

In [0]:
# seed for the pseudo-random generator
np.random.seed(1233)

n = 20  # number of data points

# regressor
x = np.linspace(-1, 1, n)  
# noise
sigma = 0.1  # std of the noise
eps = np.random.normal(loc=0, scale=sigma, size=n)
# response
y = lin_fun(x) + eps

# plot
plt.plot(x, lin_fun(x), label='true fun');  # original linear function
plt.scatter(x, y, label='noisy data');      # data affected by noise
plt.legend();

## Parameter estimation

Assume to have the dataset $\{(x_1,y_1),(x_2,y_2),\dots,(x_n,y_n)\}$, but not to know the system model.
Let's try to recover the parameters from the noisy data by solving:
$$
\hat\beta={\rm arg}\min_{\beta} \sum_{i=1}^n \lVert f(x_i;\beta) -y_i\rVert^2.
$$

### Data in compact form
We can express the function $f(x;\beta)$ in the form
$$
f(x;\beta) = \beta_1 x + \beta_0 = \left[f_1(x), f_0(1) \right] \left[\begin{array}{c}\beta_1 \\ \beta_0 \end{array} \right]  = \left[x, 1 \right] \left[\begin{array}{c}\beta_1 \\ \beta_0 \end{array} \right] 
$$
with $f_1(x) = x$ and $f_0(x)=1$.

We showed in class that the solution can be found by solving the linear system
$$
X^\top Y = X^\top X \beta 
$$
with respect to $\beta$, where we arranged the data in matrix form as follows
$$
X = \left[ 
\begin{array}{c}
x_1, 1 \\
x_2 , 1 \\
\vdots \\
x_n, 1
\end{array}
\right] 
\in \mathbb{R}^{n\times 2},
\qquad 
Y = \left[ 
\begin{array}{c}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{array}
\right] 
\in \mathbb{R}^{n}
$$

### Estimated parameters $\beta$
* When $X^\top X$ is invertible, the solution is
$$\hat\beta = (X^\top X)^{-1}X^\top Y = X^+ Y$$
**Notice**: _matrix $X^+=(X^\top X)^{-1} X^\top$ is called_ [pseudo-inverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse) _of $X$_.
* Otherwise, we solve with respect to $\beta$ the system
$$X^\top X\, \beta = X^\top Y.$$


In [0]:
# construct matrix X
ones_vec = np.ones((n, 1))  
x = x.reshape(n, 1)
X = np.concatenate((x, ones_vec), axis=1)
print('X = ')
print(X[:5])  # only the first 5 data points

# parameter estimation
S = X.T @ X
Sinv = np.linalg.inv(S)
beta_est = Sinv @ X.T @ y
y_est = X @ beta_est

# plot
plt.plot(x, lin_fun(x), label='true fun');  # original linear function
plt.scatter(x, y, label='noisy data');      # data affected by noise
plt.plot(x, y_est, label='est fun');  
plt.legend();

### Scikit-Learn: a library for machine learning in Python

The same procedure can be done with `sklearn` module.

In [0]:
from sklearn.linear_model import LinearRegression

# init the model
lr = LinearRegression()  
# estimate parameters
lr.fit(x, y)
beta_est = [lr.coef_[0], lr.intercept_]
print('beta = {}'.format(beta_est))
# estimated response
y_est = lr.predict(x)

# plot
plt.plot(x, lin_fun(x), label='true fun');      # original linear function
plt.scatter(x, y, label='noisy data');          # data affected by noise
plt.plot(x, y_est, label='est fun (sklearn)');  # estimate linear function
plt.legend();

<!-- # Multivariate case
Define a $d$-dimensional linear function $f : \mathbb{R}^d \rightarrow \mathbb{R}$ -->

## Bi-dimensional case $f : \mathbb{R}^2 \rightarrow \mathbb{R}$
$$f(\mathbf x; \beta) = \beta_1 x_1 + \beta_2 x_2 + \beta_0$$  


In [0]:
def lin_fun_2(x):
    y = 0.1 * x[:, 0] - 1.3 * x[:, 1] + 1
    return y

def lin_fun_2(x):
    beta = np.array([0.1, -1.3])  # coefficients
    beta0 = 1                     # intercept
    y = x @ beta + beta0
    return y

## Generic $d$-dimensional case $f : \mathbb{R}^d \rightarrow \mathbb{R}$
$$f(\mathbf x; \beta) = \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_d x_d$$


### Data in matrix form
We generate $n$ observations: $\{(\mathbf x_1,y_1),(\mathbf x_2,y_2),\dots,(\mathbf x_n,y_n)\}$
and store them in matrix form:
$$
X = \left[ 
\begin{array}{c}
\mathbf x_1^\top \\
\mathbf x_2^\top \\
\vdots \\
\mathbf x_n^\top
\end{array}
\right] 
\in \mathbb{R}^{n\times d},
\qquad 
Y = \left[ 
\begin{array}{c}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{array}
\right] 
\in \mathbb{R}^{n}
$$

**Remark 1:** _`sklearn` deals with the intercept for us, so we do not need to incorporate the '1' in $\mathbf x$. There is a flag, if you don't want it, see the_
[Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear\_model.LinearRegression.html)  

```python
class sklearn.linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
```

**Remark 2:**
_Often data is normalised before being processed. Modelled with functions $\{f_i(\cdot)\}$:_ 
$$f(x; \beta) = \beta_1 f_1(x_1) + \beta_2 f_2(x_2) + \dots + \beta_d f_d(x_d).$$



In [0]:
np.random.seed(0)
# a random d-dimensional example
d = 2  
beta = np.random.uniform(-1, 1, d)
beta0 = np.random.uniform(-1, 1)
def lin_fun_d(X):
    return X @ beta + beta0

# generate data
n = 100
X = np.random.uniform(-1, 3, size=(n, 2)) 
sigma = 0.1
eps = np.random.normal(loc=0, scale=sigma, size=n)
Y = lin_fun_d(X) + eps
print('size of X = {}'.format(X.shape))
print('size of Y = {}'.format(Y.shape))

In [0]:
from mpl_toolkits.mplot3d import Axes3D  # this is necessary for 3-d plots 

# plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d');
ax.set_xlabel('x_0')
ax.set_ylabel('x_1')
ax.set_zlabel('y')
ax.scatter(X[:, 0], X[:, 1], Y);

In [0]:
# estimate the parameters
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X, Y)
Y_est = lr.predict(X)

# plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d');
ax.scatter(X[:, 0], X[:, 1], Y, label='noisy data');
ax.plot_trisurf(X[:, 0], X[:, 1], Y_est, alpha=0.3, label='est fun');


## Dealing with multi-dimensional outputs $f : \mathbb{R}^d \rightarrow \mathbb{R}^m$

$$
\begin{cases}
y_1&= \beta_{11} x_1 + \beta_{21} x_2 + \dots + \beta_{d1} x_d + \varepsilon_1
\\
y_2&= \beta_{12} x_1 + \beta_{22} x_2 + \dots + \beta_{d2} x_d + \varepsilon_2
\\
&\vdots
\\
y_m&= \beta_{1m} x_1 + \beta_{2m} x_2 + \dots + \beta_{dm} x_d + \varepsilon_m
\end{cases}
$$
in matrix form 
$$ \mathbf y = B\, \mathbf x + \mathbf \varepsilon$$
<!-- or equivalently
$$ \mathbf y^\top = \mathbf x^\top B + \mathbf \beta_0^\top$$ -->
where
$$
\mathbf y = \left[ \begin{array}{c}
y_{1}\\
y_{2}\\
\vdots 
\\
y_{m}
\end{array}
\right]
\qquad
\mathbf x = \left[ \begin{array}{c}
x_{1}\\
x_{2}\\
\vdots 
\\
x_{d}
\end{array}
\right]
$$

$$
B = \left[ \begin{array}{c}
\beta_{11}, &\beta_{21}, &\dots, &\beta_{d1}\\
\beta_{12}, &\beta_{22}, &\dots, &\beta_{d2}\\
\vdots  & \vdots &\ddots& \vdots
\\
\beta_{1m}, &\beta_{2m}, &\dots, &\beta_{dm}
\end{array}
\right]
\qquad
\varepsilon = \left[ \begin{array}{c}
\varepsilon_{1}\\
\varepsilon_{2}\\
\vdots 
\\
\varepsilon_{m}
\end{array}
\right]
$$

## 01.B: Approximate polynomials with linear regression

$$f(x; \beta) =\beta_0 + \beta_1 x + \beta_2 x^2 + \dots + \beta_d x^d $$

<font color='gray'>__Remark:__ _This is linear in the parameters! In fact, 'linear' in 'linear regression' refers to the parameters. 
As we said, a more general case is_
$$f(x; \beta) = \beta_1 f_1(\mathbf x) + \beta_2 f_2(\mathbf x) + \dots + \beta_d f_d(\mathbf x)$$
</font>

Get back to the polynomials. We consider a regressor vector $[x, x^2,\dots,  x^d]$, up to a degree $d$ that we can decide.


In [0]:
def pol_fun(x):
    return -1 + -x - .1 * x**2 + .5*x**3

# generate data
n = 200
X = np.linspace(-1, 2, n).reshape(n,1) 
sigma = 0.3
eps = np.random.normal(loc=0, scale=sigma, size=(n,1))
Y = pol_fun(X) + eps

# create regressor
degree = 10
# Xpol = np.concatenate((X, X**2, X**3), axis=1)
from sklearn.preprocessing import PolynomialFeatures 
pol_feat = PolynomialFeatures(degree=degree, include_bias=False) 
Xpol = pol_feat.fit_transform(X)

# estimate parameter
lr = LinearRegression()
lr.fit(Xpol, Y)
Y_est = lr.predict(Xpol)

# plot results
plt.plot(X, pol_fun(X), label='true fun');
plt.scatter(X, Y, label='noisy data');
plt.plot(X, Y_est, label='est fun');
plt.legend();


## 01.C: Ridge regression

$$
\hat\beta={\rm arg}\min_{\beta} \sum_{i=1}^n \lVert f(x_i;\beta) -y_i\rVert^2_2 + \lambda \lVert \beta\rVert^2_2.
$$

In [0]:
# generate data [200 100 50 20 10]
n = 200
X = np.linspace(-1, 2, n).reshape(n,1) 
sigma = 0.3
eps = np.random.normal(loc=0, scale=sigma, size=(n,1))
Y = pol_fun(X) + eps

# create regressor
degree = 5
pol_feat = PolynomialFeatures(degree=degree, include_bias=False) 
Xpol = pol_feat.fit_transform(X)

# linear regression
lr = LinearRegression()
lr.fit(Xpol, Y)
Y_est = lr.predict(Xpol)

# ridge regression
from sklearn.linear_model import Ridge 
rid = Ridge()
rid.fit(Xpol, Y)
Y_rid_est = rid.predict(Xpol)

# plot results
plt.plot(X, pol_fun(X), label='true fun')
plt.scatter(X, Y, label='noisy data')
plt.plot(X, Y_est, label='lin.reg.')
plt.plot(X, Y_rid_est, label='ridge reg.')
plt.legend()

# estimated beta
print('beta_lr  = {}'.format(lr.intercept_ + lr.coef_[0]))
print('beta_rid = {}'.format(rid.intercept_ + rid.coef_[0]))

## 01.D: Exercise: Boston house price

In [0]:
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.DESCR)
X = boston.data
y = boston.target

In [0]:
# Try yourself!

## 01.E: Exercise: Lasso regression 
* __Use the documentation__: [https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
* Test on Boston house-price dataset

In [0]:
# Try yourself!

## 01.F: More on linear regression
### Confidence intervals for the parameters

Assume that $X^\top X$ is invertible, then

$$
\hat \beta = X^+Y \sim N\big(\beta, \sigma^2 (X^\top X)^{-1}\big)
$$

$$
E[\hat \beta] = E[X^+Y] = X^+E[Y] = X^+ X\beta = (X^\top X)^{-1}X^\top X \beta = \beta 
$$

$$
Var[\hat \beta] = Var[X^+Y] = X^+Var[Y] (X^+)^\top = \sigma^2 (X^\top X)^{-1} X^\top X (X^\top X)^{-1} = \sigma^2 (X^\top X)^{-1} 
$$

### A rule of thumb

* Extract the diagonal from $\sigma^2 (X^\top X)^{-1}$, which gives you an idea of the variance of each component of $\beta$.
* For each component $\beta_i$, check if the interval $(\beta_i - 2\sigma_i, \beta_i + 2\sigma_i)$ contains the zero; is such case, we are not very confident that the $\beta_i\ne 0$, thus that $x_i$ is relevant in the model.


### More formally

Knowing that the distribution of $\hat \beta$ is a (multivariate) Gaussian distribution, we can obtain a confidence interval:
$$
CI = \{\hat \beta\ s.t.\ d(\hat\beta, \beta)^2 < \chi^2_p(1-\alpha)\}
$$
such that $P(\hat \beta \in CI) = 1-\alpha$, and where
$$
d(\hat\beta, \beta)^2 = (\hat\beta - \beta)^\top \sigma^{-2} X^\top X (\hat \beta - \beta)
$$
